## Understanding the shops in the test file and in sales data

I wanted to get a grip on which of the shops we have to produce predictions for (i.e. those in the `test.csv` file) appear in the sales data and to get some grasp on the presence of relevant sales data for these shops. In real life shops may open or close and some shops may sell more items than others. To get this understanding I developed the following chart (should be inline below) and this notebook describes how it was built.

![image.png](foo.png)

There are a number of components to this chart:

* At the top is a list of the `shop_id`s that appear in the `sales_train.csv` data (57 in total - see notes on cleaning later in notebook)

* At the bottom is the list of the `shop_id`s that appear in the `test.csv` file (42 in total)

* The `shop_id`s are ordered according to the total item sales for that shop for the last month in the sales data (Oct 2015) with the shop with the most sales on the left.

* The bars in the lower half of the chart indicate the volume of sales by shop in that last month relative to the left-most shop.
    - _These bars are coloured blue if the shop is associated with a city or green if an online store (see later)_
    
    
* The top half of the chart includes indicators showing the presence or absence of sales data over different time intervals for each shop:

    - _The top (dark red) bar indicates that there was at least one sale prior to October 2014 (1 year prior to end of sales data)_
    - _The next (red) bar indicates there were sales between October 2014 and May 2015_
    - _The yellow bar indicates the presence of sales for that shop in the last 6 months (i.e. between May 2015 and October 2015)_
    - _The green bar indicates that shop has recorded sales in the last month._
    - _Note: the height of these bars is fixed, it doesn't try to show us anything about the volume of sales in these periods_
   
There has been some clean up of the data in producing this chart which is discussed in the section on constructing the chart below.

## Insights taken from the chart

* The shops in the test data have all had sales in the month prior to the month we have to make predictions for. 

* Shops without sales in the last month do not appear in the test set. It looks like 13 stores have closed.

* The exception to this is `shop_id = 9` and `shop_id = 20` which had data in the last month but do not appear in the test set. In each of these cases there seems to be missing data between 1 year and 6 months ago. We could speculate that this is the reason they are not included and think about whether to include them in any training examples. The shop meta data for `shop_id = 9` described this is shop as 'Outbound Trade' which may indicate it is unusual, but `shop_id=20` seems like a store in Moscow.  

* One shop (`shop_id=36`) looks relatively new with sales data only in last 6 months.

* Two of the shop names could not be matched to a city. `shop_id = 12` and `shop_id = 55` are described as `Emergency Shop Online` and `Digital warehouse 1C-Online` respectively. Whilst at the higher end of the sales ranking the total volume of sales from these shops in the last month doesn't seem unusual compared to stores whos name contain a city, which I assume are physical stores.

* A couple of the shops `shop_id = 25` and `shop_id = 31` seem to sell significantly more than other stores.


## Building the chart

The chart is made using matplotlib `axvline` to draw the bars.

In [ ]:
import sys
import itertools
from pathlib import Path 
competition_data = Path('../input/competitive-data-science-predict-future-sales')
additional_data = Path('../input/russian-cities-population-wikipedia')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, Markdown

Load in the shops in sales and test

In [ ]:
# The shops and items in the test set for which we must make predictions
shops_in_test=pd.read_csv(competition_data/'test.csv',usecols=['shop_id']).shop_id.unique()

# The sales data
daily_sales=pd.read_csv(competition_data/'sales_train.csv',usecols=['shop_id','date_block_num','item_cnt_day'])

### Version 1: Which of these shops are in the sales data?

The first version of the chart orders the shops by their `shop_id` and shows the presence or absence of data in sales and test sets. It is wrapped up in the function `plot_shop_trace`:

In [ ]:

def plot_shop_trace(sales_data):
    
    all_sales_shops=sales_data.shop_id.unique()
    sales_by_shop_grid = sales_data.groupby(['shop_id','date_block_num']).sum().reset_index().pivot(index='shop_id',columns='date_block_num',values='item_cnt_day')
    sales_by_shop_l6m = sales_by_shop_grid.iloc[:,-6:].sum(axis=1)
    sales_by_shop_l6m2y = sales_by_shop_grid.iloc[:,-12:-6].sum(axis=1)
    sales_by_shop_y1 = sales_by_shop_grid.iloc[:,0:-12].sum(axis=1)

    yr1_shops = sales_by_shop_l6m[sales_by_shop_y1 > 0]
    yr2_shops = sales_by_shop_l6m[sales_by_shop_l6m2y > 0]
    live_shops = sales_by_shop_l6m[sales_by_shop_l6m > 0]

    f,ax=plt.subplots(1,1,figsize=(16,8))

    [ax.text(_x-0.22, 0.95, str(_x)) for _x in all_sales_shops];

    ax.text(60,0.8,"Shops with sales between prior to Oct 2014")
    [ax.axvline(_x, linewidth=4, ymin=0.7, ymax=0.9, color='#ac0f00') for _x in yr1_shops.index.unique()];

    ax.text(60,0.63,"Shops with sales between Oct 2014 and May 2015")
    [ax.axvline(_x, linewidth=4, ymin=0.6, ymax=0.67, color='#fc4f30') for _x in yr2_shops.index.unique()];

    ax.text(60,0.53,"Shops with sales in last 6 months")
    [ax.axvline(_x, linewidth=4, ymin=0.5, ymax=0.57, color='#e5ae38') for _x in live_shops.index.unique()];

    ax.text(60,0.3,"Shops we need to make predictions for")
    [ax.axvline(_x, linewidth=4, ymin=0.2, ymax=0.4, color='#30a2da') for _x in shops_in_test];

    [ax.text(_x-0.22, 0.1, str(_x)) for _x in shops_in_test];

    ax.set_axis_off(); 
    return ax

plot_shop_trace(daily_sales);

### Version 2: Improving the information density using meta-data and sales volumes

#### Using meta-data to clean up the sales data

The file `shops.csv` contains information about each shop - i.e. it's 'name'. 

Some of these details suggest that some of the `shop_id`s are duplicated and we should fix the sales record accordingly. Redrawing the plot after these changes we lose a couple of the `shop_ids` which have been combined.

In [ ]:
# shops:
# some shops look near duplicates from name...
# shop_id                    shop_name
# 10                         Жуковский ул. Чкалова 39м?
# 11                         Жуковский ул. Чкалова 39м²
# shop_id=10 is in the test set, so make occurances of 11 into 10s
daily_sales['shop_id']=daily_sales.shop_id.apply(lambda id: 10 if id==11 else id)

# combination of names and consecutive sales record suggests the following are same shop
# choose the later label to align with later months.
daily_sales['shop_id']=daily_sales.shop_id.apply(lambda id: 57 if id==0 else id)
daily_sales['shop_id']=daily_sales.shop_id.apply(lambda id: 58 if id==1 else id)

# redraw the trace plot
plot_shop_trace(daily_sales)

#### Using meta-data to derive shop type.

The shop names often start with the name of a city. Except for a few cases which look like they could be 'virtual' stores of some kind. ('_Outbound Trade_', '_Emergency Shop Online_','_Digital warehouse 1C-Online_'). Are these exceptional stores in the test set? Perhaps the sales volumes from these stores will be very different?

I use the `shop_name` information to label the shops as either 'city' (which I assume are bricks-and-mortar stores) or other. This is done by trying to match parts of the shop names to known russian cities. There was some hand-tuning required.

In [ ]:
def extract_city_name(s):
    return s.lower().replace('!','').replace('. ', '.').replace(',','').replace('”','').replace('“','').replace('\"','').strip().split(' ')[0]

shop_meta=pd.read_csv(competition_data/'shops.csv')
shop_meta['extracted_city'] = shop_meta.shop_name.apply(lambda n: extract_city_name(n))

# a table taken from wikipedia - list of russian cities
# https://en.wikipedia.org/wiki/List_of_cities_and_towns_in_Russia_by_population
ru_cities=pd.read_csv(additional_data/'russian_city_populations.csv')
ru_cities['matching_city']=ru_cities.Russian.apply(lambda s: s.strip().lower())
shop_meta = shop_meta.merge(ru_cities[['matching_city','Population(2010 Census)[3]']], how='left', left_on='extracted_city', right_on='matching_city')  

# There are some that are not found. Some of them are translation errors that can be fixed manually

# 'интернет-магазин = 'online store'
# 'сергиев = is a city Sergiyev Posad (space problem)
# 'спб = spb
# 'н.новгород is a city Nizhny Novgorod (abbr problem) => Нижний Новгород
# 'выездная = 'exit'
# 'цифровой = 'digital'
# 'адыгея = The Republic of Adygea - assume captial city Майкоп
# 'ростовнадону' = is a city Rostov-on-Don]
# 'СПб' --> abbr for St Petersburg

# 3 of them do not seem to translate to city locations like the rest. Here are the literal translations:
special_shop_translations = {
    "Выездная Торговля":"Outbound Trade",
    "Интернет-магазин ЧС":"Emergency Shop Online",
    "Цифровой склад 1С-Онлайн":"Digital warehouse 1C-Online"
}

# fix the cities not found
mislabelled_cities={'спб':'санкт-петербург','сергиев':'Сергиев Посад','н.новгород':'Нижний Новгород','адыгея':'Майкоп','ростовнадону':'Ростов-на-Дону'}
shop_meta['matching_city']=shop_meta.apply(lambda r: mislabelled_cities[r['extracted_city']].lower() if r['extracted_city'] in mislabelled_cities.keys() else r['matching_city'], axis=1).astype('category')

# add a column to indicate whether we think this is a 'city' / 'normal' shop, or, one of the 'special' shops
shop_meta['shop_type']=shop_meta.apply(lambda r: 'city' if type(r['matching_city'])==str else special_shop_translations[r['shop_name']], axis=1)
r={'city':0,'Outbound Trade':1,'Emergency Shop Online':2,'Digital warehouse 1C-Online':3};
shop_meta['shop_type_numeric']=shop_meta.shop_type.apply(lambda s: r[s]).astype('category')

del shop_meta['extracted_city']; del shop_meta['Population(2010 Census)[3]']


I now need to adjust the chart plotting code to use the shop meta data to label city and other store types.

I also wanted to add some information about sales volumes to the chart, to understand if all stores in the test set sold similar volumes. I decided I could use the height of the bars indicating that a store was in the test set to communicate the volume of sales in the last month.

To bring make the chart clearer I ordered the shopd by volume of sales in the last month rather than by `_id`. These changes are incorprated into a second version of the `plot_shop_trace` function:

In [ ]:
def plot_shop_trace2(sales_data, fs=(16,8)):
    
    all_sales_shops=sales_data.shop_id.unique()
    sales_by_shop_grid = sales_data.groupby(['shop_id','date_block_num']).sum().reset_index().pivot(index='shop_id',columns='date_block_num',values='item_cnt_day')
    sales_by_shop_l1m = sales_by_shop_grid.iloc[:,-1:].sum(axis=1)
    sales_by_shop_l6m = sales_by_shop_grid.iloc[:,-6:].sum(axis=1)
    sales_by_shop_l6m2y = sales_by_shop_grid.iloc[:,-12:-6].sum(axis=1)
    
    sales_by_shop_y1 = sales_by_shop_grid.iloc[:,0:-12].sum(axis=1)

    yr1_shops = sales_by_shop_l6m[sales_by_shop_y1 > 0]
    yr2_shops = sales_by_shop_l6m[sales_by_shop_l6m2y > 0]
    live_shops = sales_by_shop_l6m[sales_by_shop_l6m > 0]
    vlive_shops = sales_by_shop_l6m[sales_by_shop_l1m > 0]

    ranked=sales_by_shop_l1m.sort_values(ascending=False).reset_index().reset_index()
    ranked.columns=['ranking','shop_id','sales']
    ranked=ranked.set_index('shop_id')
    
    FS=12
    
    f,ax=plt.subplots(1,1,figsize=fs)

    #ax.text(58,0.95,"shop_ids:  in training data (sales_train.csv)",fontsize=FS)
    [ax.axvline(ranked.loc[_x].ranking, linewidth=1, ymin=-0.1, ymax=0.95, color='#eeeeee') for _x in all_sales_shops];
    [ax.text(ranked.loc[_x].ranking-(0.25*len(str(_x))), 0.95, str(_x),fontsize=12) for _x in all_sales_shops];

    #ax.text(61,0.8,"with sales between prior to Oct 2014",fontsize=FS)
    [ax.axvline(ranked.loc[_x].ranking, linewidth=4, ymin=0.7, ymax=0.9, color='#ac0f00') for _x in yr1_shops.index.unique()];

    #ax.text(61,0.63,"with sales between Oct 2014 and May 2015",fontsize=FS)
    [ax.axvline(ranked.loc[_x].ranking, linewidth=4, ymin=0.6, ymax=0.68, color='#fc4f30') for _x in yr2_shops.index.unique()];

    #ax.text(61,0.55,"with sales in last 6 months",fontsize=FS)
    [ax.axvline(ranked.loc[_x].ranking, linewidth=4, ymin=0.53, ymax=0.58, color='#e5ae38') for _x in live_shops.index.unique()];

    #ax.text(61,0.48,"with sales in last month",fontsize=FS)
    [ax.axvline(ranked.loc[_x].ranking, linewidth=4, ymin=0.49, ymax=0.51, color='g') for _x in vlive_shops.index.unique()];

    #ax.text(58,0.0,"shop_ids we need to make predictions for",fontsize=FS)
    shop_meta['stc']=shop_meta.shop_type_numeric.apply(lambda v: '#30a2da' if v==0 else '#6d904f' )
    rel_sales=(sales_by_shop_l1m.sort_values()/sales_by_shop_l1m.max()).clip(0,100000)
    [ax.axvline(ranked.loc[_x].ranking, linewidth=4, ymin=0.0, ymax=0.4*rel_sales.loc[_x], color=shop_meta.set_index('shop_id').loc[_x,'stc']) for _x in shops_in_test];

    [ax.text(ranked.loc[_x].ranking-(0.25*len(str(_x))), -0.075, str(_x), fontsize=12) for _x in shops_in_test];
    ax.set_axis_off(); 
    
    return ax

In [ ]:
ax=plot_shop_trace2(daily_sales, fs=(20,6))
ax.set_title("Mapping shop in test item file to shops in sales data.\n",fontsize=16);
plt.savefig('foo.png')

And that's that. Hope you have found this useful.